# **Pulmonary Embolism**

**Essentials**
Pulmonary Embolism is a life threatening condition in which a blood clot (thrombus) becomes lodged in an artery in the lung and blocks blood flow to the lung. The clots typically originates from the deep venous system of the lower extremities as shown in the *image below*. After traveling to the lung, large thrombi can lodge at the bifurcation of the main pulmonary artery or the lobar branches and cause hemodynamic compromise. The condition usually presents with an abrupt onset of sharp chest pain, shortness of breath, and hypoxia. However symptoms may vary from sudden catastrophic hemodynamic collapse to gradually progressive dyspnea. 

**Diagnosis**
There are various tools for diagnosing PE. 
In this project we will focus on CT angiography commonly used in clinical practice. Intravenous contrast is administered through the patient's veins just prior to obtaining the CT, and the CT scan is timed to obtain images when the contrast is primarily in the pulmonary blood vessels. Note: this is where quality issues can occur. Sometimes the timing of the scan is off, resulting in a suboptimal scan. This can result in non-diagnostic or equivocal studies. In addition, if the patient is moving a lot or has metal in their body, this can cause artifacts which may obscure a clot.


(Image Source: Medscape)
![](https://img.medscapestatic.com/pi/meds/ckb/22/38722tn.jpg)

# **Pulmonary Embolism CT Scan**

In the image below, notice the areas marked in the red color in the **Pulmonary Trunk**. These darker gray spots are evidence of pulmonary embolism in the pulmonary trunk. This is a type of Central Pulmonary Embolism.

![](https://imgur.com/eUWTVwU.jpg)

**Import Libraries**

In [1]:
#Libraries
import numpy as np #numpy offers mathematical function(functions, linear algebra etc.)
import pandas as pd #Data manipulation and analysis
import tensorflow as tf #tensorflow platform for machine learning.
import os #provides functions for interactiong with operating systems.
import gc #Manages memory used by Python objects.
import time 
from IPython.display import clear_output #clear output of a cell
from tensorflow.keras.models import load_model #Loads a model saved
from tensorflow.keras.callbacks import ModelCheckpoint as MC #used in conjunction with training using model.fit() to save a model or weights (in a checkpoint file) 
from tensorflow.keras import backend as K #offers high-level building blocks that are useful to develop deep learning models
from tensorflow import keras #Python-based framework that makes it easy to debug and explore. 
from tensorflow.keras.models import Model #represents the actual neural network model. 
import pydicom# Python package for working with DICOM files

# Dense implements the operation
# Droputout Dropout consists in randomly setting a fraction rate of input units to 0 at each update during training time, which helps prevent overfitting.
#Conv2D applies a 2D convolutionover an inout signal
#LeakyRelu is a type of activation function based on a RELU
from tensorflow.keras.layers import Input, Dense,Dropout, Conv2D, LeakyReLU 

In [2]:
#ignore warnings
import warnings
warnings.filterwarnings('ignore')


# **Dataset**

The following are description of the dataset used in this notebook's dataset:

1. **Test** - contains UIDs of all testing images directory
2. **Train** - contains UIDs of all labels of the training images  directory 
3. **Sample** Submission - contains rows for each UID+label combination that requires a prediction. 

**Loading Data**


In [3]:
#Path to the dataframe
root = '/kaggle/input/rsna-str-pulmonary-embolism-detection'

# os.listdir() method in python is used to get the list of all files and directories in the specified directory.
for item in os.listdir(root):
    path = os.path.join(root, item)
    if os.path.isfile(path):
        print(path)

In [4]:
#view the csv file containing the training data from the path generated above  
print('Train data...')

train = pd.read_csv("/kaggle/input/rsna-str-pulmonary-embolism-detection/train.csv")
print(train.shape)
train.head()

In [5]:
#view the csv file containing the testing data from the path generated above  
print('Test data...')
test = pd.read_csv("/kaggle/input/rsna-str-pulmonary-embolism-detection/test.csv")
print(test.shape)
test.head()

In [6]:
print('Sample data...')
sample = pd.read_csv("/kaggle/input/rsna-str-pulmonary-embolism-detection/sample_submission.csv")
print(sample.shape)
sample.head()

In [7]:
#confirm the test shape
DEBUG = (test.shape[0]==146853)
DEBUG

In [8]:
#Remove the duplicates from the test dataframe.
test = pd.read_csv('/kaggle/input/rsna-str-pulmonary-embolism-detection/test.csv')

test_study = test.drop_duplicates('StudyInstanceUID')[['StudyInstanceUID','SeriesInstanceUID']]

if DEBUG:
    test_study = test_study.head(25)
test_study.shape                                                      

In [9]:
test_study.head(30)

In [10]:
# check for duplicated values in the from the test_study dataframe.
test_study.duplicated()

In [11]:
train.info()

Checking all IDs from the sample dataset to make sure 
that we'll be making the correct training targets.

In [12]:
#the ID function returns the identity (unique integer) of an object.
ids = sample.id
counter = [1 for _ in range(10)]
mapper = []

for i in ids:
    #Split a string into a list where each word is a list item:
    n = '_'.join(i.split('_')[1:])
    if n not in mapper:
        mapper.append(n)
    else:
        counter[mapper.index(n)]+=1
        
print("List of Keys:")
print(mapper, sep='\n')
print()
print("Count of items per key:")
print(counter)

In [13]:
#progressbar decorator for iterators.
from tqdm.notebook import tqdm

prediction_counts ={}
for idx in tqdm(range(sample.shape[0])):
    if len(sample['id'][idx][13:])>1:
        key = sample['id'][idx][13:]
    else:
        key = 'pe_present_on_image'
    prediction_counts[key]= prediction_counts.get(key, 0) + 1
print(f'total row count in submission: {sample.shape[0]}')
prediction_counts

# **DICOM Visualisation**

The Visualization Toolkit is an open-source software system for 3D computer graphics, image processing and scientific visualization. In this notebook we use the VTK library to view the DICOM images (DICOM (Digital Imaging and Communications in Medicine) is a standard protocol for the management and transmission of medical images). Reading data in VTK is really easy and pleasant, but unfortunately also problematic. This is because this library is focused on visualization, not DICOM processing.

In [14]:
#import the vtk library
import vtk
#Convert NumPy arrays to VTK arrays 
from vtk.util import numpy_support
#OpenCV (CV2) is an open source computer vision and machine learning software library.
import cv2

#Read the Dicom images
reader = vtk.vtkDICOMImageReader()

#define a function 
def get_img(path):
    
    #specify the path of the data file
    reader.SetFileName(path) 
    # update it
    reader.Update()
    
    #Calculates the max extent which includes all of the features.
    # Load dimensions using `GetDataExtent`
    _extent= reader.GetDataExtent()
    ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]

    # Load spacing values
    ConstPixelSpacing = reader.GetPixelSpacing()
    
   # Load all the pixel data into an appropriate sized NumPy array named ArrayDicom:
    imagedata= reader.GetOutput()
    
    #Get the 'vtkPointData' object from the 'vtkImageData' object
    pointdata= imagedata.GetPointData()
    
    #Gets the array data needed 
    arraydata = pointdata.GetArray(0)
    
    #converts the "Arraydata" to a numpy array 
    ArrayDicom = numpy_support.vtk_to_numpy(arraydata)
    
    #reshape the Numpy array to a 3D using "ConsPixelDims"
    ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')
    
    #resize it
    ArrayDicom = cv2.resize(ArrayDicom,(512,512))
    return ArrayDicom

In [15]:
import matplotlib.pyplot as plt

#view a sample DICOM file
fpath = "/kaggle/input/rsna-str-pulmonary-embolism-detection/train/0003b3d648eb/d2b2960c2bbf/00ac73cfc372.dcm"

#get the image data
ds = get_img(fpath)

func = lambda x:int((2**15+ x)*(255/2**16))

int16_to_uint8= np.vectorize(func)

def show_dicom_images(dcom):
    f, ax = plt.subplots(1,2, figsize = (16,20))
    data_row_img = int16_to_uint8(ds)
    #Set the colormap to 'bone'.
    ax[0].imshow(data_row_img, cmap=plt.cm.bone)
    ax[1].imshow(ds,cmap=plt.cm.bone)
    
    
    #print(data_row_img)
    ax[0].axis("off")
    ax[0].set_title('8-bit Dicom Image')
    ax[1].axis('off')
    ax[1].set_title('16-bit Dicom Image')
    plt.show()
        
show_dicom_images(ds)

In [16]:
#fetch metadata info from DICOM file
dcm_file =  pydicom.read_file("../input/rsna-str-pulmonary-embolism-detection/train/0003b3d648eb/d2b2960c2bbf/00ac73cfc372.dcm")
print(dcm_file.file_meta)

# **DICOM Visualization**
Monai is a PyTorch-based, open-source framework for deep learning in healthcare imaging used to create state-of-the-art, end-to-end training workflows.

In [17]:
#we install the current release by running the following command:
!pip install monai

In [18]:
#The array is zoomed using spline interpolation of the requested order
from scipy.ndimage.interpolation import zoom
#The glob module is used to retrieve files/pathnames matching a specified pattern.
from glob import glob
#This module implements some useful functions on pathnames.
import os, os.path as osp
import torch.nn.functional as F

def load_dicom_array(f):
    #find all the pathnames matching a specified pattern
    dicom_files = glob(osp.join(f, '*.dcm'))
    
    #Reads and returns a DICOM dataset stored in the DICOM File Format.
    dicoms = [pydicom.dcmread(d) for d in dicom_files]
    
    #The rescale slope and rescale intercept allow to transform the pixel values 
    # to HU or other units.
    M = float(dicoms[0].RescaleSlope)
    B = float(dicoms[0].RescaleIntercept)
    
    # Assume all images are axial
    #he x, y, and z coordinates of the upper left hand corner of the image
    z_pos = [float(d.ImagePositionPatient[-1]) for d in dicoms]
    dicoms = np.asarray([d.pixel_array for d in dicoms])
    dicoms = dicoms[np.argsort(z_pos)]
    dicoms = dicoms * M
    dicoms = dicoms + B
    return dicoms, np.asarray(dicom_files)[np.argsort(z_pos)]

def window(img, WL=50, WW=350):
    upper, lower = WL+WW//2, WL-WW//2
    X = np.clip(img.copy(), lower, upper)
    X = X - np.min(X)
    X = X / np.max(X)
    X = (X*255.0).astype('uint8')
    return X

def read_dicom(dcm_path, image_size=256):
    image, files = load_dicom_array(dcm_path)
    #Expand the shape of an array.
    image_lung = np.expand_dims(window(image, WL=-600, WW=1500), axis=3)
    image_mediastinal = np.expand_dims(window(image, WL=40, WW=400), axis=3)
    image_pe_specific = np.expand_dims(window(image, WL=100, WW=700), axis=3)
    #combine the above 3
    image = np.concatenate([image_mediastinal, image_pe_specific, image_lung], axis=3)
    rat = image_size / np.max(image.shape[1:])
    image = zoom(image, [1.,rat,rat,1.], prefilter=False, order=1)
    return image

In [19]:
#torch the samples and their corresponding labels
from torch.utils.data import Dataset
#An interface for handling random state locally, currently based on a class variable
from monai.transforms import Randomizable
from monai.transforms import apply_transform

class RSNADataset3D(Dataset, Randomizable):
    def __init__(self, csv, mode, transform=None):
        self.csv = csv.reset_index()
        self.mode = mode
        self.transform = transform
    def __len__(self):
        return self.csv.shape[0]
    def randomize(self) -> None:
        #Get the info of the Maximum value of the given dtype.
        MAX_SEED = np.iinfo(np.uint32).max + 1
        self._seed = self.R.randint(MAX_SEED, dtype="uint32")    
    def __getitem__(self, index):
        self.randomize()
        row = self.csv.iloc[index]
        try:
            img = read_dicom(os.path.join('/kaggle/input/rsna-str-pulmonary-embolism-detection/test', row.StudyInstanceUID, row.SeriesInstanceUID))
        except:
            img = np.zeros((144, 256, 256, 3),dtype=np.uint8)
            
        # (144, 256, 256, 3)  Z, H, W, ch
        #transpose() function transpose index and columns of the dataframe. 
        #It reflect the DataFrame over its main diagonal by writing rows as columns and vice-versa.
        img = img[:,:,:,::-1].transpose(3,1,2,0) # -> ch, H, W, Z
        if self.transform is not None:
            # returns True if the specified object is of the specified type,
            if isinstance(self.transform, Randomizable):
                self.transform.set_random_state(seed=self._seed)
            img = apply_transform(self.transform, img)
        if self.mode == 'test':
            return img  

In [20]:
image_size = 160
out_dim = 9
batch_size = 2

In [21]:
#The transform method is callable that processes data.
from monai.transforms import Compose, ScaleIntensity, Resize, ToTensor
#To Tensor: Converts the input image to a tensor without applying any other transformation
#Resize: Resize the input image to given spatial size
#Compose: provides the ability to chain a series of callables together in a sequential manner.
#ScaleIntensity: Scale the intensity of input image to the given value range
val_transforms = Compose([ScaleIntensity(),Resize((image_size, image_size, image_size)),ToTensor()])

In [22]:
import torchvision.transforms as transforms
from pylab import rcParams

if DEBUG:
    dataset_show = RSNADataset3D(test_study.head(5), 'test', transform=val_transforms)
    #modify multiple settings in a single group at once
    rcParams['figure.figsize'] = 12,5
    for i in range(3):
        f, axarr = plt.subplots(1,3)
        img = dataset_show[i]
        print(img.shape)
        for j in range(3):            
            axarr[j].imshow(img.numpy().transpose(1,2,3,0).mean(axis=j))        
            axarr[j].set_title(i)
        plt.show()

**Pydicom** is a pure Python package for working with DICOM files. It lets you read, modify and write DICOM data in an easy "pythonic" way. **Scikit-image** is a collection of algorithms for image processing. 

In [23]:
#load pydicom module
import pydicom as dcm

#Apply a modality lookup table or rescale operation to arr.
from pydicom.pixel_data_handlers.util import apply_modality_lut

#Clear objects connected to the label image border.
from skimage.segmentation import clear_border

#Find the edge magnitude using Roberts’ cross operator.
from skimage.filters import roberts, sobel

#Label connected regions of an integer array.
#Regionprop: Measure properties of labeled image regions
from skimage.measure import label, regionprops

#Disk:Generates a flat, disk-shaped footprint.
#Binary_erosion: Returns a fast binary morphological erosion of an image.
#Dilation: Return grayscale morphological dilation of an image.
from skimage.morphology import disk, binary_erosion, binary_closing, dilation

**Image Animation**

In [24]:
def get_segmented_lungs(im2, plot = False):
    im = im2.copy()
    binary = im<-400
    
    if plot:
        plt.imshow(binary)
        plt.show()
        
    cleared = clear_border(binary)
    
    if plot:
        plt.imshow(cleared)
        plt.show()
    
    label_image = label(cleared)
    
    if plot:
        plt.imshow(label_image)
        plt.show()
        
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    if len(areas)>0:
        for region in regionprops(label_image):
            if region.area< areas[0]:
                for coordinates in region.coords:
                    lable_image[coordinates[0], coordinates[1]]=0
                    binary = lable_image>0
                    
    if plot:
        plt.imshow(binary)
        plt.show()
        
        
    selem = disk(2)
    binary = binary_erosion(binary, selem)
    
    selem = disk(10)
    binary = binary_closing(binary,selem)
    
    if plot:
        plt.imshow(binary)
        plt.show()
        
    edges = roberts(binary)
    
    if plot:
        plt.imshow(binary)
        plt.show()
        
    selem = disk(4)
    binary = dilation(binary, selem)
    get_high_vals = binary ==0
    im[get_high_vals]= - 2000
    
    if plot:
        plt.imshow(im)
        plt.show()
    
    return im, binary
    

In [25]:
train = pd.read_csv("/kaggle/input/rsna-str-pulmonary-embolism-detection/train.csv")
data_path = "/kaggle/input/rsna-str-pulmonary-embolism-detection/train/"
studyID,seriesID,SOPID= train.loc[50,['StudyInstanceUID','SeriesInstanceUID','SOPInstanceUID']].values

one_series_path = data_path + studyID + "/" +seriesID+ "/" 
one_series = []
one_series_seg=[]

for i in os.listdir(one_series_path):
    dicom_path= one_series_path+"/"+i
    img=dcm.dcmread(dicom_path)
    img_data = img.pixel_array
    hu = apply_modality_lut(img_data, img)
    img_seg, _ = get_segmented_lungs(hu)
    length = int(img.InstanceNumber)
    one_series.append((length,img_data))
    one_series_seg.append((length, img_seg))
    
one_series.sort()
one_series_seg.sort()
one_series_seg = [s[1] for s in one_series_seg]
one_series = [s[1] for s in one_series]

In [26]:
#load animation module
from matplotlib import animation, rc
rc('animation', html = "jshtml")

def animate(ims, ims_seg):
    fig,(ax1,ax2)= plt.subplots(1,2, figsize = (15,8))
    ax1.axis("off")
    ax2.axis("off")
    im = ax1.imshow(ims[0])
    im2 = ax2.imshow(ims_seg[0])
   

    def animate_func(i):
        im.set_data(ims[i])
        im2.set_data(ims_seg[i])
        return [im,im2]
    
    anim = animation.FuncAnimation(fig, animate_func, frames= len(ims), interval = 1000/24)
    
    return anim

In [27]:
pulmonary_embolism = animate(one_series,one_series_seg)

In [28]:
pulmonary_embolism